In [1]:
# Importing Dependencies
import pandas as pd
import requests
import time
from datetime import datetime, timedelta

#Import API Key
from api_key import wx_api_key

In [2]:
# Creating List of Dates
dates = []

# Assuming 365 Days in a Year
number_of_days = 365

# Inputting Start Date
start_date_obj = datetime(2015, 1, 1)

# For Loop to Create List of Dates
for day in range(number_of_days):
    new_date = start_date_obj + timedelta(day)
    new_date = new_date.strftime("%Y-%m-%d")
    dates.append(new_date)

In [ ]:
# Reading in CSV File
filepath = '../Resources/lg_arprts.csv'
f = pd.read_csv(filepath)
df = pd.DataFrame(f)
df = df.drop(columns=['Unnamed: 0']) # Dropping Column
iata_codes = df['iata_code']

iata_code = []

for code in iata_codes:
    iata_code.append(code)

###  Perform API Calls

In [2]:
# Base URL
base_url = "https://api.worldweatheronline.com/premium/v1/past-weather.ashx?"

# WorldWeather Online Has Two Formats with XML as the Default
format_output = 'json'

# Time Interval, Set to 6 Hourly Intervals
tp = 6

# Include location
incLoc = 'yes'

# Building Query URL
query_url = f"{base_url}key={wx_api_key}&format={format_output}&tp={tp}&includelocation={incLoc}"

In [4]:
# Creating Empty Lists
date_list = []
time_list = []
lat =[]
lng = []
tempF = []
DewPointF = []
windspeedMiles = []
WindGustMiles = []
winddir16Point = []
weatherCode = []
weatherDesc = []
precipInches = []
humidity = []
visibilityMiles = []
pressure = []
cloudcover = []
HeatIndexF = []
WindChillF = []
totalSnow_cm = []

# Using a For Loop to Make Successive API Calls
print('Beginning Data Retrieval')
print('------------------------')

# Loop Through Each Time Interval (*Hard Coded for 6 Hourly Data*)
for time in range(4):
    # Loop Through Dates        
    for date in dates:
        # Making Requests
        response = requests.get(f'{query_url}&date={date}&q={loc}').json()
        # Using Try/Except to Grab Variables if Available
        try:
            print(f'Processing Time Interval {time} for {date} for {loc}')
            date_list.append(response['data']['weather'][0]['date'])
            totalSnow_cm.append(response['data']['weather'][0]['totalSnow_cm'])
            time_list.append(response['data']['weather'][0]['hourly'][time]['time'])
            lat.append(response['data']['nearest_area'][0]['latitude'])
            lng.append(response['data']['nearest_area'][0]['longitude'])
            tempF.append(response['data']['weather'][0]['hourly'][time]['tempF'])
            DewPointF.append(response['data']['weather'][0]['hourly'][time]['DewPointF'])
            windspeedMiles.append(response['data']['weather'][0]['hourly'][time]['windspeedMiles'])
            WindGustMiles.append(response['data']['weather'][0]['hourly'][time]['WindGustMiles'])
            winddir16Point.append(response['data']['weather'][0]['hourly'][time]['winddir16Point'])
            weatherCode.append(response['data']['weather'][0]['hourly'][time]['weatherCode'])
            weatherDesc.append(response['data']['weather'][0]['hourly'][time]['weatherDesc'][0]['value'])
            precipInches.append(response['data']['weather'][0]['hourly'][time]['precipInches'])
            humidity.append(response['data']['weather'][0]['hourly'][time]['humidity'])
            visibilityMiles.append(response['data']['weather'][0]['hourly'][time]['visibilityMiles'])
            pressure.append(response['data']['weather'][0]['hourly'][time]['pressure'])
            cloudcover.append(response['data']['weather'][0]['hourly'][time]['cloudcover'])
            HeatIndexF.append(response['data']['weather'][0]['hourly'][time]['HeatIndexF'])
            WindChillF.append(response['data']['weather'][0]['hourly'][time]['WindChillF'])
        except KeyError:
            print('Day not found. Skipping...')
print('------------------------')
print('Data Retrieval Complete')
print('------------------------')

Beginning Data Retrieval
------------------------
Processing Time 0 for 2015-01-01
Processing Time 0 for 2015-01-02
Processing Time 0 for 2015-01-03
Processing Time 0 for 2015-01-04
Processing Time 0 for 2015-01-05
Processing Time 0 for 2015-01-06
Processing Time 0 for 2015-01-07
Processing Time 0 for 2015-01-08
Processing Time 0 for 2015-01-09
Processing Time 0 for 2015-01-10
Processing Time 0 for 2015-01-11
Processing Time 0 for 2015-01-12
Processing Time 0 for 2015-01-13
Processing Time 0 for 2015-01-14
Processing Time 0 for 2015-01-15
Processing Time 0 for 2015-01-16
Processing Time 0 for 2015-01-17
Processing Time 0 for 2015-01-18
Processing Time 0 for 2015-01-19
Processing Time 0 for 2015-01-20
Processing Time 0 for 2015-01-21
Processing Time 0 for 2015-01-22
Processing Time 0 for 2015-01-23
Processing Time 0 for 2015-01-24
Processing Time 0 for 2015-01-25
Processing Time 0 for 2015-01-26
Processing Time 0 for 2015-01-27
Processing Time 0 for 2015-01-28
Processing Time 0 for 2015

Processing Time 0 for 2015-09-05
Processing Time 0 for 2015-09-06
Processing Time 0 for 2015-09-07
Processing Time 0 for 2015-09-08
Processing Time 0 for 2015-09-09
Processing Time 0 for 2015-09-10
Processing Time 0 for 2015-09-11
Processing Time 0 for 2015-09-12
Processing Time 0 for 2015-09-13
Processing Time 0 for 2015-09-14
Processing Time 0 for 2015-09-15
Processing Time 0 for 2015-09-16
Processing Time 0 for 2015-09-17
Processing Time 0 for 2015-09-18
Processing Time 0 for 2015-09-19
Processing Time 0 for 2015-09-20
Processing Time 0 for 2015-09-21
Processing Time 0 for 2015-09-22
Processing Time 0 for 2015-09-23
Processing Time 0 for 2015-09-24
Processing Time 0 for 2015-09-25
Processing Time 0 for 2015-09-26
Processing Time 0 for 2015-09-27
Processing Time 0 for 2015-09-28
Processing Time 0 for 2015-09-29
Processing Time 0 for 2015-09-30
Processing Time 0 for 2015-10-01
Processing Time 0 for 2015-10-02
Processing Time 0 for 2015-10-03
Processing Time 0 for 2015-10-04
Processing

Processing Time 1 for 2015-05-12
Processing Time 1 for 2015-05-13
Processing Time 1 for 2015-05-14
Processing Time 1 for 2015-05-15
Processing Time 1 for 2015-05-16
Processing Time 1 for 2015-05-17
Processing Time 1 for 2015-05-18
Processing Time 1 for 2015-05-19
Processing Time 1 for 2015-05-20
Processing Time 1 for 2015-05-21
Processing Time 1 for 2015-05-22
Processing Time 1 for 2015-05-23
Processing Time 1 for 2015-05-24
Processing Time 1 for 2015-05-25
Processing Time 1 for 2015-05-26
Processing Time 1 for 2015-05-27
Processing Time 1 for 2015-05-28
Processing Time 1 for 2015-05-29
Processing Time 1 for 2015-05-30
Processing Time 1 for 2015-05-31
Processing Time 1 for 2015-06-01
Processing Time 1 for 2015-06-02
Processing Time 1 for 2015-06-03
Processing Time 1 for 2015-06-04
Processing Time 1 for 2015-06-05
Processing Time 1 for 2015-06-06
Processing Time 1 for 2015-06-07
Processing Time 1 for 2015-06-08
Processing Time 1 for 2015-06-09
Processing Time 1 for 2015-06-10
Processing

Processing Time 2 for 2015-01-16
Processing Time 2 for 2015-01-17
Processing Time 2 for 2015-01-18
Processing Time 2 for 2015-01-19
Processing Time 2 for 2015-01-20
Processing Time 2 for 2015-01-21
Processing Time 2 for 2015-01-22
Processing Time 2 for 2015-01-23
Processing Time 2 for 2015-01-24
Processing Time 2 for 2015-01-25
Processing Time 2 for 2015-01-26
Processing Time 2 for 2015-01-27
Processing Time 2 for 2015-01-28
Processing Time 2 for 2015-01-29
Processing Time 2 for 2015-01-30
Processing Time 2 for 2015-01-31
Processing Time 2 for 2015-02-01
Processing Time 2 for 2015-02-02
Processing Time 2 for 2015-02-03
Processing Time 2 for 2015-02-04
Processing Time 2 for 2015-02-05
Processing Time 2 for 2015-02-06
Processing Time 2 for 2015-02-07
Processing Time 2 for 2015-02-08
Processing Time 2 for 2015-02-09
Processing Time 2 for 2015-02-10
Processing Time 2 for 2015-02-11
Processing Time 2 for 2015-02-12
Processing Time 2 for 2015-02-13
Processing Time 2 for 2015-02-14
Processing

Processing Time 2 for 2015-09-22
Processing Time 2 for 2015-09-23
Processing Time 2 for 2015-09-24
Processing Time 2 for 2015-09-25
Processing Time 2 for 2015-09-26
Processing Time 2 for 2015-09-27
Processing Time 2 for 2015-09-28
Processing Time 2 for 2015-09-29
Processing Time 2 for 2015-09-30
Processing Time 2 for 2015-10-01
Processing Time 2 for 2015-10-02
Processing Time 2 for 2015-10-03
Processing Time 2 for 2015-10-04
Processing Time 2 for 2015-10-05
Processing Time 2 for 2015-10-06
Processing Time 2 for 2015-10-07
Processing Time 2 for 2015-10-08
Processing Time 2 for 2015-10-09
Processing Time 2 for 2015-10-10
Processing Time 2 for 2015-10-11
Processing Time 2 for 2015-10-12
Processing Time 2 for 2015-10-13
Processing Time 2 for 2015-10-14
Processing Time 2 for 2015-10-15
Processing Time 2 for 2015-10-16
Processing Time 2 for 2015-10-17
Processing Time 2 for 2015-10-18
Processing Time 2 for 2015-10-19
Processing Time 2 for 2015-10-20
Processing Time 2 for 2015-10-21
Processing

Processing Time 3 for 2015-05-29
Processing Time 3 for 2015-05-30
Processing Time 3 for 2015-05-31
Processing Time 3 for 2015-06-01
Processing Time 3 for 2015-06-02
Processing Time 3 for 2015-06-03
Processing Time 3 for 2015-06-04
Processing Time 3 for 2015-06-05
Processing Time 3 for 2015-06-06
Processing Time 3 for 2015-06-07
Processing Time 3 for 2015-06-08
Processing Time 3 for 2015-06-09
Processing Time 3 for 2015-06-10
Processing Time 3 for 2015-06-11
Processing Time 3 for 2015-06-12
Processing Time 3 for 2015-06-13
Processing Time 3 for 2015-06-14
Processing Time 3 for 2015-06-15
Processing Time 3 for 2015-06-16
Processing Time 3 for 2015-06-17
Processing Time 3 for 2015-06-18
Processing Time 3 for 2015-06-19
Processing Time 3 for 2015-06-20
Processing Time 3 for 2015-06-21
Processing Time 3 for 2015-06-22
Processing Time 3 for 2015-06-23
Processing Time 3 for 2015-06-24
Processing Time 3 for 2015-06-25
Processing Time 3 for 2015-06-26
Processing Time 3 for 2015-06-27
Processing

### Create DataFrame

In [5]:
 # Creating DataFrame
historical_wx_df = pd.DataFrame({
    'Date': date_list,
    'Time': time_list,
    'Latitude': lat,
    'Longitude': lng,
    'Weather Code' : weatherCode,
    'Weather Description': weatherDesc,
    'Temperature (F)': tempF,
    'Heat Index (F)': HeatIndexF,
    'Wind Chill (F)': WindChillF,
    'Dew Point (F)': DewPointF,
    'Wind Speed (mph)': windspeedMiles,
    'Wind Gusts (mph)': WindGustMiles,
    'Wind Direction': winddir16Point,
    'Precipitation (in)': precipInches,
    'Humidity (%)': humidity,
    'Visibility (mi)': visibilityMiles,
    'Pressure (mb)': pressure,
    'Cloud Coverage (%)': cloudcover,
    'Snow Total (cm)': totalSnow_cm
})

### Creating CSV File

In [6]:
# Creating CSV File
historical_wx_df.to_csv('../Resources/historical_wx.csv')

### Reading CSV File

In [7]:
 # Reading in CSV File
filepath = '../Resources/historical_wx.csv'
f = pd.read_csv(filepath)
df = pd.DataFrame(f)
df = df.drop(columns=['Unnamed: 0']) # Dropping Column

df.head()

,Date,Time,Latitude,Longitude,Weather Code,Weather Description,Temperature (F),Heat Index (F),Wind Chill (F),Dew Point (F),Wind Speed (mph),Wind Gusts (mph),Wind Direction,Precipitation (in),Humidity (%),Visibility (mi),Pressure (mb),Cloud Coverage (%),Snow Total (cm)
0,2015-01-01,0,40.714,-74.006,113,Clear,27,28,19,15,10,14,WSW,0.0,59,6,1022,10,0.0
1,2015-01-02,0,40.714,-74.006,113,Clear,35,36,27,24,13,21,WSW,0.0,63,6,1018,6,0.0
2,2015-01-03,0,40.714,-74.006,122,Overcast,32,33,26,25,8,14,NNE,0.0,74,6,1032,100,0.0
3,2015-01-04,0,40.714,-74.006,143,Mist,43,44,42,43,3,5,N,0.0,97,1,1018,100,0.0
4,2015-01-05,0,40.714,-74.006,113,Clear,43,43,36,38,13,22,W,0.0,83,6,1011,3,0.0
